In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
max_length = 256

In [3]:
from transformers import AutoTokenizer

encoding_args = {
    "max_length" : max_length,
    "padding" : True,
    "truncation" : True,
    "return_tensors" : "pt"
}

tokenizer = AutoTokenizer.from_pretrained("/raid/m13519061/ta/facebook-absa/src/output/fewshot/checkpoint-5700")

/home/m13519061/anaconda3/envs/absa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import torch
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda:0


In [5]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("/raid/m13519061/ta/facebook-absa/src/output/fewshot/checkpoint-5700")
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(250112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (w

In [6]:
import data_utils
from evaluation import compute_metrics, summary_score

catch_answer_fn = data_utils.AnswerCatcher().lego_absa
decoding_args = {
    "skip_special_tokens" : False
}

In [7]:
from tqdm import tqdm
from typing import List, Dict, Tuple

def generate_predictions(model,tokenizer,data,device=torch.device("cuda:0"),decoding_args:Dict={}) -> List[str]:
    # Predict
    model = model
    tokenizer = tokenizer
    tensor_predictions = []
    with torch.no_grad():
        for text in tqdm(data):
            input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)
            tensor_predictions.extend(model.generate(input_ids=input_ids, pad_token_id=tokenizer.pad_token_id,eos_token_id=tokenizer.eos_token_id,max_length=max_length).cpu())
            input_ids = input_ids.cpu()
            # attention_mask = attention_mask.cpu()
    tensor_predictions = [[token for token in row if token != -100] for row in tensor_predictions]
    predictions = tokenizer.batch_decode(tensor_predictions,**decoding_args)
    predictions = [el for el in predictions]
    return predictions

In [8]:
from datasets import Dataset

test_path = "../data/absa/id/william/test.txt"

test_tasks = [
    {
        "paradigm" : "extraction",
        "se_order" : "oas",
        "method" : "lego_absa"
    }
]

test = data_utils.read_data(test_path)
test_ds = data_utils.data_gen(data=test, nt_se_order="aos", tasks=test_tasks, n_fold=1, algo="round_robin", shuffle=False)
test_ds = Dataset.from_list(test_ds)

100%|██████████| 1000/1000 [00:00<00:00, 19940.78it/s]


In [9]:
str_preds = generate_predictions(model, tokenizer, test_ds["input"], device, decoding_args)

100%|██████████| 995/995 [08:13<00:00,  2.02it/s]


In [10]:
str_preds = [el.replace("</s>",'').replace("<pad>",'') for el in str_preds]

In [11]:
oas_ext_preds = [catch_answer_fn(out, "oas", text) for out, text in zip(str_preds, test_ds["input"])]

In [12]:
targets = [catch_answer_fn(out, "oas", text) for out, text in zip(test_ds["output"], test_ds["input"])]

In [13]:
summary_score(oas_ext_preds,targets)

{'recall': 0.7371343946686413,
 'precision': 0.8000799680127949,
 'f1_score': 0.7673184394125225}

In [14]:
few_prompter = data_utils.FewShotPrompter()

In [15]:
from copy import deepcopy
fs_test = [
    few_prompter.lego_absa(el["text"],t,"oas",99) for el,t in zip(test,oas_ext_preds)
]
str_preds = generate_predictions(model, tokenizer, fs_test, device, decoding_args)
str_preds = [el.replace("</s>",'').replace("<pad>",'') for el in str_preds]
oas_few_preds1 = [catch_answer_fn(out, "oas", text) for out, text in zip(str_preds, test_ds["input"])]
oas_com_preds1 = []
for el1, el2 in zip(oas_ext_preds,oas_few_preds1):
    result = deepcopy(el1)
    for el in el2:
        if el not in result:
            result.append(el)
    oas_com_preds1.append(result)

100%|██████████| 995/995 [09:41<00:00,  1.71it/s]


In [16]:
summary_score(oas_com_preds1,targets)

{'recall': 0.7404664938911514,
 'precision': 0.43525335643135554,
 'f1_score': 0.5482437448048108}

In [18]:
cnt1 = 0
for el in oas_ext_preds:
    cnt1 += len(el)

cnt2 = 0
for el in oas_com_preds1:
    cnt2 += len(el)

In [19]:
cnt1

2501

In [20]:
cnt2

4618

In [27]:
oas_com_preds1[2]

[{'opinion': 'terjangkau', 'aspect': 'harga', 'sentiment': 'positive'},
 {'opinion': 'nyaman', 'aspect': 'fasilitas', 'sentiment': 'positive'}]

In [28]:
oas_ext_preds[2]

[{'opinion': 'terjangkau', 'aspect': 'harga', 'sentiment': 'positive'},
 {'opinion': 'nyaman', 'aspect': 'fasilitas', 'sentiment': 'positive'}]

In [34]:
beda = []
for i in range(len(oas_com_preds1)):
    if oas_ext_preds[i] != oas_com_preds1[i]:
        beda.append(i)

In [36]:
beda[:5]

[19, 45, 56, 62, 67]

In [37]:
oas_ext_preds[45]

[{'opinion': 'lama', 'aspect': 'bangunan', 'sentiment': 'negative'},
 {'opinion': 'kurang bersih', 'aspect': 'bangunan', 'sentiment': 'negative'},
 {'opinion': 'mengalir terus',
  'aspect': 'kamar mandinya airnya',
  'sentiment': 'negative'},
 {'opinion': 'berisik',
  'aspect': 'kamar mandinya airnya',
  'sentiment': 'negative'},
 {'opinion': 'tidak banyak pilihan',
  'aspect': 'sarapannya',
  'sentiment': 'negative'},
 {'opinion': 'tidak ramah', 'aspect': 'recepsionis', 'sentiment': 'negative'}]

In [38]:
oas_com_preds1[45]

[{'opinion': 'lama', 'aspect': 'bangunan', 'sentiment': 'negative'},
 {'opinion': 'kurang bersih', 'aspect': 'bangunan', 'sentiment': 'negative'},
 {'opinion': 'mengalir terus',
  'aspect': 'kamar mandinya airnya',
  'sentiment': 'negative'},
 {'opinion': 'berisik',
  'aspect': 'kamar mandinya airnya',
  'sentiment': 'negative'},
 {'opinion': 'tidak banyak pilihan',
  'aspect': 'sarapannya',
  'sentiment': 'negative'},
 {'opinion': 'tidak ramah', 'aspect': 'recepsionis', 'sentiment': 'negative'},
 {'opinion': 'kecewa', 'aspect': 'NULL', 'sentiment': 'negative'}]

In [39]:
targets[45]

[{'opinion': 'lama', 'aspect': 'bangunan', 'sentiment': 'negative'},
 {'opinion': 'kurang bersih', 'aspect': 'bangunan', 'sentiment': 'negative'},
 {'opinion': 'mengalir terus',
  'aspect': 'kamar mandinya airnya',
  'sentiment': 'negative'},
 {'opinion': 'berisik',
  'aspect': 'kamar mandinya airnya',
  'sentiment': 'negative'},
 {'opinion': 'masih tabung', 'aspect': 'tv', 'sentiment': 'negative'},
 {'opinion': 'kecewa', 'aspect': 'fasilitas', 'sentiment': 'negative'},
 {'opinion': 'tidak banyak pilihan',
  'aspect': 'sarapannya',
  'sentiment': 'negative'},
 {'opinion': 'sudah dingin', 'aspect': 'makan', 'sentiment': 'negative'},
 {'opinion': 'tidak ramah', 'aspect': 'recepsionis', 'sentiment': 'negative'},
 {'opinion': 'tidak sesuai ekpektasi',
  'aspect': 'NULL',
  'sentiment': 'negative'}]